In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from refuge.config import load_config
from refuge.training import main

In [ ]:
cfg = load_config()
cfg

In [ ]:
cfg.project.name

In [ ]:
from transformers import Adafactor, GPTNeoXTokenizerFast

from refuge._vendor.mkultra.soft_prompt import SoftPrompt
from refuge._vendor.mkultra.tuning import GPTNeoXPromptTuningLM

In [ ]:
model_name = "databricks/dolly-v2-3b"

tokenizer = GPTNeoXTokenizerFast.from_pretrained(model_name, padding_side="left")
model = GPTNeoXPromptTuningLM.from_pretrained(model_name, device_map="auto")

In [ ]:
model.get_soft_params()

In [ ]:
model.learned_embedding.shape

In [ ]:
model.gpt_neox.embed_in.weight.shape

In [ ]:
tokenizer.encode("hello!")

In [ ]:
hello_embedding = model.gpt_neox.embed_in.weight[[25521, 2], :]
hello_embedding

In [ ]:
sp = SoftPrompt.from_string("hello!", model, tokenizer)
sp.get_inputs_embeds()

In [ ]:
main(20)